In [3]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import ks_2samp, kruskal, ttest_ind, t, kstwo, wilcoxon
from utils_best_arm import calc_return
from tqdm import tqdm
import numpy as np
import seaborn as sns

In [9]:
symbols = os.listdir("./Results/")
path1 = f"./Results/{symbols[0]}/exp1_concat/"
rewards = os.listdir(path1)
indexes_df = []

bh_df = []
for symbol in symbols[:]:
    path1 = f"./Results/{symbol}/exp1_concat/{rewards[0]}"
    df = pd.read_csv(path1).fillna(0)

    bh_percentage = df.iloc[:, 0].values
    test_portfolios = {0: df.iloc[:, 0].values}
    indexes = calc_return(bh_percentage, test_portfolios)
    bh_df.append(indexes)

bh_df = pd.concat(bh_df, axis=1).T
bh_df.columns = [f"B&H_{col.replace('_test', '')}" for col in bh_df.columns]
bh_df = pd.DataFrame(np.tile(bh_df, [100, 1]), columns=bh_df.columns)
bh_df.reset_index(drop=True, inplace=True)


for reward in tqdm(rewards[:]):
    ls2 = []
    for symbol in symbols:
        path1 = f"./Results/{symbol}/exp1_concat/{reward}"
        df = pd.read_csv(path1).fillna(0)

        bh_percentage = df.iloc[:, 0].values
        seeds = list(df.columns)[1:]
        ls = []
        for seed in seeds:
            test_portfolios = {seed: df.loc[:, seed].values}
            indexes = calc_return(bh_percentage, test_portfolios)
            ls.append(indexes)

        ls_df = pd.concat(ls, axis=1).T
        ls_df.columns = [f"{reward.replace('_0.csv', '')}_{col}" for col in ls_df.columns]
        ls2.append(ls_df)
    
    ls2 = pd.concat(ls2)
    indexes_df.append(ls2)


indexes_df = pd.concat(indexes_df, axis=1)
indexes_df.reset_index(drop=True, inplace=True)
indexes_df = pd.concat([bh_df, indexes_df], axis=1)
indexes_df.to_csv("./ts-run-results/[exp1]data-for-wilcoxon-test.csv", index=False)
indexes_df

100%|██████████| 7/7 [07:12<00:00, 61.80s/it]


,future_profit_10_sortino_test,future_profit_10_sharpe_test,future_profit_10_risk_test,future_profit_10_mdd_test,future_profit_10_downrisk_test,future_profit_10_cumreturn_test,future_profit_15_sortino_test,future_profit_15_sharpe_test,future_profit_15_risk_test,future_profit_15_mdd_test,...,old_sharpe_risk_test,old_sharpe_mdd_test,old_sharpe_downrisk_test,old_sharpe_cumreturn_test,profit_sortino_test,profit_sharpe_test,profit_risk_test,profit_mdd_test,profit_downrisk_test,profit_cumreturn_test
0,-0.194454,-0.140018,0.285617,0.365095,0.205661,-0.077662,3.724984,2.359298,0.222742,0.124035,...,0.259814,0.339939,0.196637,0.118868,3.871673,2.448116,0.221965,0.124035,0.140352,0.679396
1,0.792093,0.553986,0.252632,0.202028,0.176690,0.114202,-0.156210,-0.112363,0.285712,0.365095,...,0.000000,0.000000,0.000000,0.000000,3.152395,2.094754,0.237061,0.189043,0.157526,0.597161
2,3.947286,2.619400,0.235946,0.125227,0.156573,0.803427,3.596624,2.328678,0.230898,0.138673,...,0.195186,0.083299,0.110738,0.630728,2.380883,1.637781,0.248816,0.227177,0.171158,0.457045
3,0.036609,0.026857,0.284875,0.365095,0.208987,-0.032446,-0.178812,-0.128638,0.285672,0.365095,...,0.114543,0.056506,0.040947,0.210670,2.910500,1.947060,0.232029,0.189043,0.155222,0.529085
4,2.699224,1.845974,0.247155,0.190147,0.169027,0.530335,3.282796,2.167468,0.234116,0.163514,...,0.203649,0.439999,0.223731,-0.430048,3.021448,2.001666,0.233776,0.189043,0.154873,0.553345
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1.857424,1.238086,0.179865,0.150697,0.119891,0.229330,0.377017,0.296515,0.219075,0.178580,...,0.142124,0.359449,0.189360,-0.323227,5.719206,3.295122,0.163183,0.073345,0.094018,0.688655
996,2.984290,1.938625,0.184067,0.150697,0.119572,0.404549,0.000000,1.001990,0.006161,0.000000,...,0.000224,0.000224,0.000000,-0.000224,2.460955,1.854947,0.198992,0.178580,0.149990,0.417609
997,-0.630630,-0.516299,0.245517,0.280761,0.201006,-0.145459,2.076654,1.409774,0.191322,0.150697,...,0.182419,0.494235,0.203364,-0.463159,4.285209,2.711073,0.174954,0.099822,0.110686,0.581931
998,3.083182,2.000792,0.183045,0.130598,0.118785,0.418060,2.039587,1.374839,0.197424,0.150697,...,0.012874,0.012900,0.000000,-0.012900,4.102009,2.585474,0.174047,0.099822,0.109701,0.544237


In [14]:
def replace_reward(name):
    if "future_profit" in name:
        return f"FP{name.split('_')[2]}_{name.split('_')[3]}"
    elif "old_sharpe" in name:
        return f"PSR_{name.split('_')[-1]}"
    elif "old_profit" in name:
        return f"PP1_{name.split('_')[-1]}"
    elif "profit" == name.split("_")[0]:
        return f"FP5_{name.split('_')[-1]}"
    else:
        return name

dt = {
    "old_sharpe_0": "PSR",
    "old_profit_0": "PP1",
    "profit_0": "FP5",
    "future_profit_1_0": "FP1",
    "future_profit_10_0": "FP10",
    "future_profit_15_0": "FP15",
    "future_profit_20_0": "FP20"
}

replace_name = lambda x: dt[x.replace(".csv", "")]

cols = [col.replace("_test", "") for col in indexes_df.columns]
indexes_df.columns = list(map(replace_reward, cols))
indexes_df

,B&H_sortino,B&H_sharpe,B&H_risk,B&H_mdd,B&H_downrisk,B&H_cumreturn,FP10_sortino,FP10_sharpe,FP10_risk,FP10_mdd,...,PSR_risk,PSR_mdd,PSR_downrisk,PSR_cumreturn,FP5_sortino,FP5_sharpe,FP5_risk,FP5_mdd,FP5_downrisk,FP5_cumreturn
0,-0.157004,-0.112971,0.285712,0.365095,0.205582,-0.070543,-0.194454,-0.140018,0.285617,0.365095,...,0.259814,0.339939,0.196637,0.118868,3.871673,2.448116,0.221965,0.124035,0.140352,0.679396
1,0.836190,0.643763,0.236477,0.155024,0.182059,0.132149,0.792093,0.553986,0.252632,0.202028,...,0.000000,0.000000,0.000000,0.000000,3.152395,2.094754,0.237061,0.189043,0.157526,0.597161
2,0.034455,0.025707,0.240779,0.207193,0.179644,-0.022560,3.947286,2.619400,0.235946,0.125227,...,0.195186,0.083299,0.110738,0.630728,2.380883,1.637781,0.248816,0.227177,0.171158,0.457045
3,0.732692,0.484028,0.312351,0.246703,0.206344,0.107767,0.036609,0.026857,0.284875,0.365095,...,0.114543,0.056506,0.040947,0.210670,2.910500,1.947060,0.232029,0.189043,0.155222,0.529085
4,-0.573555,-0.420436,0.324648,0.331021,0.237978,-0.172746,2.699224,1.845974,0.247155,0.190147,...,0.203649,0.439999,0.223731,-0.430048,3.021448,2.001666,0.233776,0.189043,0.154873,0.553345
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1.308166,0.970799,0.338935,0.247834,0.251526,0.311847,1.857424,1.238086,0.179865,0.150697,...,0.142124,0.359449,0.189360,-0.323227,5.719206,3.295122,0.163183,0.073345,0.094018,0.688655
996,0.922114,0.659294,0.263219,0.174759,0.188196,0.148917,2.984290,1.938625,0.184067,0.150697,...,0.000224,0.000224,0.000000,-0.000224,2.460955,1.854947,0.198992,0.178580,0.149990,0.417609
997,-0.531712,-0.384728,0.240386,0.214923,0.173935,-0.114382,-0.630630,-0.516299,0.245517,0.280761,...,0.182419,0.494235,0.203364,-0.463159,4.285209,2.711073,0.174954,0.099822,0.110686,0.581931
998,0.118557,0.085178,0.205550,0.149467,0.147679,-0.003678,3.083182,2.000792,0.183045,0.130598,...,0.012874,0.012900,0.000000,-0.012900,4.102009,2.585474,0.174047,0.099822,0.109701,0.544237


In [ ]:
rewards2 = list(map(replace_name, rewards))

criteria = ["sortino", "sharpe", "risk", "mdd", "downrisk", "cumreturn",]
test_df = []
for c in criteria:
    cols = [col for col in indexes_df.columns if c == col.split("_")[-1]]

    tmp_df = indexes_df[cols]

    n = tmp_df.shape[0] * 2 - 2

    vars = list(tmp_df.columns)

    for var1 in vars:
        if var1 in rewards: continue
        
        for _var in vars:
            if _var == var1:
                continue

            test = wilcoxon(tmp_df[var1], tmp_df[_var])

            _var1 = var1.split("_")[0]
            _var2 = _var.split("_")[-1]

            median1 = round(tmp_df[var1].median(), 4)
            median2 = round(tmp_df[_var].median(), 4)

            if _var2 in ["cumreturn", "sortino", "sharpe"]:
                if median1 > median2:
                    sign = 1
                else:
                    sign = 0
            else:
                if median1 < median2:
                    sign = 1
                else:
                    sign = 0
        

            test_df.append({
                "reward1": _var1,
                "var": _var2,
                "reward2": f"{_var.split('_')[0]}",
                "statistic": test.statistic,
                "pvalue": round(test.pvalue, 4),
                "median1": median1,
                "median2": median2,
                "median1 > median2": sign,
            })


test_df = pd.DataFrame(test_df)
test_df = test_df.sort_values("reward1").reset_index(drop=True)
test_df["var"] = test_df["var"].replace({
    "cumreturn": "CR",
    "downrisk": "DR",
    "mdd": "MDD",
    "risk": "Risk",
    "sharpe": "SR",
    "sortino": "STR",
})


test_df.to_csv("./ts-run-results/[exp1]wilcoxon-test.csv", index=False)
test_df

In [5]:
import pandas as pd
import numpy as np

In [28]:
test_df = pd.read_csv("./ts-run-results/[exp1]wilcoxon-test.csv",)

In [29]:
def convert_sig(x):
    if x < 0.01:
        return "***"
    elif 0.01 <= x < 0.05:
        return "**"
    elif 0.05 <= x < 0.1:
        return "*"
    else:
        return ""


def convert_star(x):
    if x == 1:
        return "√" 
    elif np.isnan(x):
        return "-"
    else:
        return ""


def construct_CR(var):
    print(var)
    tmp_df = test_df[test_df["var"] == var]
    CR_df1 = pd.pivot_table(tmp_df, values="statistic", index="reward1", columns="reward2").round(0).astype(str).applymap(lambda x: "-" if x == "nan" else x)
    CR_df1 = CR_df1.applymap(lambda x: x.replace(".0", ""))
    CR_df2 = pd.pivot_table(tmp_df, values="pvalue", index="reward1", columns="reward2").round(2)
    CR_df3 = CR_df2.applymap(convert_sig)
    CR_df2 = CR_df2.astype(str)
    CR_df4 = pd.pivot_table(tmp_df, values="median1 > median2", index="reward1", columns="reward2")
    CR_df4 = CR_df4.applymap(convert_star)
    CR_df = CR_df1 + "^{" + CR_df3 + "}(" + CR_df4 + ")"
    CR_df = CR_df.applymap(lambda x: x.replace("()", ""))
    CR_df = CR_df.applymap(lambda x: "-" if "-^" in x else x)
    
    CR_df.index.name = "reward"
    columns = ['B&H', 'PP1', 'PSR', 'FP1', 'FP5', 'FP10', 'FP15', 'FP20', ]
    CR_df = CR_df[columns].T[columns].T

    CR_df.to_csv(f"./ts-run-results/[exp1]wilcoxon-test-criteria-{var}.csv")

    return CR_df


vars = test_df["var"].unique().tolist()

for var in vars:
    construct_CR(var)


STR
CR
SR
DR
MDD
Risk


In [30]:
var = vars[1]
CR_df = construct_CR(var)
CR_df

CR


reward2,B&H,PP1,PSR,FP1,FP5,FP10,FP15,FP20
reward,,,,,,,,
B&H,-,52623^{***}(√),249834^{},48927^{***},28874^{***},42014^{***},76428^{***},142419^{***}
PP1,52623^{***},-,75962^{***},3180^{***},708^{***},1013^{***},2388^{***},6889^{***}
PSR,249834^{}(√),75962^{***}(√),-,78054^{***},58276^{***},67936^{***},93024^{***},141220^{***}
FP1,48927^{***}(√),3180^{***}(√),78054^{***}(√),-,197592^{***},215684^{**},187702^{***}(√),110348^{***}(√)
FP5,28874^{***}(√),708^{***}(√),58276^{***}(√),197592^{***}(√),-,209552^{***}(√),128644^{***}(√),66095^{***}(√)
FP10,42014^{***}(√),1013^{***}(√),67936^{***}(√),215684^{**}(√),209552^{***},-,137280^{***}(√),60654^{***}(√)
FP15,76428^{***}(√),2388^{***}(√),93024^{***}(√),187702^{***},128644^{***},137280^{***},-,104289^{***}(√)
FP20,142419^{***}(√),6889^{***}(√),141220^{***}(√),110348^{***},66095^{***},60654^{***},104289^{***},-
